In [1]:
import pyspark.sql.functions as F
import pandas as pd 
import numpy as np
from pydataset import data
from  pyspark.sql.functions import col, expr
from  pyspark.sql.functions import *
from pyspark.sql.functions import regexp_extract, regexp_replace


import pyspark.sql.functions as F



In [2]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark.range(5).show()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/24 08:39:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [3]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()


In [4]:
import multiprocessing
import pyspark

nprocs = multiprocessing.cpu_count()

spark = (pyspark.sql.SparkSession.builder
 .master('local')
 .config('spark.jars.packages', 'mysql:mysql-connector-java:8.0.16')
 .config('spark.driver.memory', '4G')
 .config('spark.driver.cores', nprocs)
 .config('spark.sql.shuffle.partitions', nprocs)
 .appName('MySparkApplication')
 .getOrCreate())


23/08/24 08:39:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


1. Create a spark data frame that contains your favorite programming languages.

    * The name of the column should be language
    * View the schema of the dataframe
    * Output the shape of the dataframe
    * Show the first 5 records in the dataframe


In [5]:


pd_df = pd.DataFrame({'language': ['python', 'r',
 'scala', 'java', 'c', 'c++']})



In [6]:


df = spark.createDataFrame(pd_df)



In [7]:


df.schema



StructType([StructField('language', StringType(), True)])

In [8]:


df.count(), len(df.columns)



(6, 1)

In [9]:


df.show(5)



+--------+
|language|
+--------+
|  python|
|       r|
|   scala|
|    java|
|       c|
+--------+
only showing top 5 rows



2. Load the mpg dataset as a spark dataframe.

In [11]:
mpg_pandas = data('mpg')

In [12]:


mpg = spark.createDataFrame(mpg_pandas)



In [13]:
mpg.select(
    F.concat(
    F.lit('The '), 
    mpg.year,
    F.lit(' '), 
    mpg.manufacturer,
    F.lit(' has a '),
    mpg.cyl,
    F.lit(' cylinder engine')).alias('description')
    ).show(truncate=False)


+------------------------------------------+
|description                               |
+------------------------------------------+
|The 1999 audi has a 4 cylinder engine     |
|The 1999 audi has a 4 cylinder engine     |
|The 2008 audi has a 4 cylinder engine     |
|The 2008 audi has a 4 cylinder engine     |
|The 1999 audi has a 6 cylinder engine     |
|The 1999 audi has a 6 cylinder engine     |
|The 2008 audi has a 6 cylinder engine     |
|The 1999 audi has a 4 cylinder engine     |
|The 1999 audi has a 4 cylinder engine     |
|The 2008 audi has a 4 cylinder engine     |
|The 2008 audi has a 4 cylinder engine     |
|The 1999 audi has a 6 cylinder engine     |
|The 1999 audi has a 6 cylinder engine     |
|The 2008 audi has a 6 cylinder engine     |
|The 2008 audi has a 6 cylinder engine     |
|The 1999 audi has a 6 cylinder engine     |
|The 2008 audi has a 6 cylinder engine     |
|The 2008 audi has a 8 cylinder engine     |
|The 2008 chevrolet has a 8 cylinder engine|
|The 2008 

In [14]:


mpg.select(mpg.trans).show(10)



+----------+
|     trans|
+----------+
|  auto(l5)|
|manual(m5)|
|manual(m6)|
|  auto(av)|
|  auto(l5)|
|manual(m5)|
|  auto(av)|
|manual(m5)|
|  auto(l5)|
|manual(m6)|
+----------+
only showing top 10 rows



In [15]:
mpg.withColumn('tran', 
               F.when(
                mpg.trans.like('auto%'), 'auto'
                ).otherwise(
                    'manual'
                    )
            ).select('trans','tran').show(5)

+----------+------+
|     trans|  tran|
+----------+------+
|  auto(l5)|  auto|
|manual(m5)|manual|
|manual(m6)|manual|
|  auto(av)|  auto|
|  auto(l5)|  auto|
+----------+------+
only showing top 5 rows



3. load the tips dataset as a spark dataframe

In [16]:
tips = spark.createDataFrame(data('tips'))

In [17]:


tips.count()



244

In [18]:


tips.groupby('smoker').count().show()



+------+-----+
|smoker|count|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+



In [19]:
tips.groupby('smoker').count().withColumn(
    'percent', F.round(
        F.col('count') / tips.count() * 100
        )
        ).show()


+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|   62.0|
|   Yes|   93|   38.0|
+------+-----+-------+



In [20]:
tips.withColumn(
    'tip_percentage', tips.tip / tips.total_bill
    ).show(5)


+----------+----+------+------+---+------+----+-------------------+
|total_bill| tip|   sex|smoker|day|  time|size|     tip_percentage|
+----------+----+------+------+---+------+----+-------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|0.05944673337257211|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|0.16054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|0.16658733936220846|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 0.1397804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|0.14680764538430255|
+----------+----+------+------+---+------+----+-------------------+
only showing top 5 rows



In [21]:
tips.select(
    tips.tip,
    tips.total_bill,
    F.round(
        (tips.tip / tips.total_bill), 4
        ).alias('tip_percentage')
        ).show(5)

+----+----------+--------------+
| tip|total_bill|tip_percentage|
+----+----------+--------------+
|1.01|     16.99|        0.0594|
|1.66|     10.34|        0.1605|
| 3.5|     21.01|        0.1666|
|3.31|     23.68|        0.1398|
|3.61|     24.59|        0.1468|
+----+----------+--------------+
only showing top 5 rows



In [22]:
tips.withColumn(
    'tip_percentage',
    tips.tip / tips.total_bill
    ).groupby(
    'sex',
    'smoker').agg(
        F.round(
            F.mean('tip_percentage'),4).alias(
                'avg_tip_p')
                ).show()


+------+------+---------+
|   sex|smoker|avg_tip_p|
+------+------+---------+
|  Male|    No|   0.1607|
|Female|    No|   0.1569|
|  Male|   Yes|   0.1528|
|Female|   Yes|   0.1822|
+------+------+---------+



In [23]:
tips.groupby(
    'sex').pivot(
        'smoker').agg(
            F.round(F.mean(tips.tip / tips.total_bill),4)).show()

+------+------+------+
|   sex|    No|   Yes|
+------+------+------+
|Female|0.1569|0.1822|
|  Male|0.1607|0.1528|
+------+------+------+



4. Use the seattle weather dataset referenced in the lesson to answer the questions below

In [24]:
from vega_datasets import data
weather = data.seattle_weather()
weather = spark.createDataFrame(weather)

In [25]:
weather.show(5)

+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [26]:
weather = \
weather.withColumn(
    "temp_max", (F.col("temp_max") * 9 / 5 + 32)
).withColumn("temp_min", (F.col("temp_min") * 9 / 5 + 32))
weather.show(5)


+-------------------+-------------+--------+--------+----+-------+
|               date|precipitation|temp_max|temp_min|wind|weather|
+-------------------+-------------+--------+--------+----+-------+
|2012-01-01 00:00:00|          0.0|   55.04|    41.0| 4.7|drizzle|
|2012-01-02 00:00:00|         10.9|   51.08|   37.04| 4.5|   rain|
|2012-01-03 00:00:00|          0.8|   53.06|   44.96| 2.3|   rain|
|2012-01-04 00:00:00|         20.3|   53.96|   42.08| 4.7|   rain|
|2012-01-05 00:00:00|          1.3|   48.02|   37.04| 6.1|   rain|
+-------------------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [27]:


weather.count(), len(weather.columns)



(1461, 6)

In [28]:
weather.withColumn(
    'month', F.month(weather.date)
    ).groupby(
    F.col('month')
    ).agg(
        F.mean(
            weather.precipitation
            ).alias('avg_rainfall')
            ).sort(
                F.col('avg_rainfall').desc()
                ).first()[0]

11

In [29]:
weather.withColumn(
    'year',
    F.year(weather.date)
    ).groupby(
    F.col('year')
    ).agg(
        F.mean(weather.wind).alias('avg_wind')
        ).sort(
            F.col('avg_wind').desc()
            ).first()

Row(year=2012, avg_wind=3.400819672131148)

In [30]:
weather.filter(
    F.month(weather.date) == 1
    ).groupby(
    weather.weather
    ).count().sort(
        F.col('count').desc()
        ).show()

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



In [31]:
(
    weather.filter(F.month("date") == 7)
    .filter(F.year("date") > 2012)
    .filter(F.year("date") < 2015)
    .filter(F.col("weather") == F.lit("sun"))
    .agg(
        F.avg("temp_max").alias("average_high_temp"),
        F.avg("temp_min").alias("average_low_temp"),
    )
    .show()
)


+-----------------+-----------------+
|average_high_temp| average_low_temp|
+-----------------+-----------------+
|80.29192307692308|57.52884615384615|
+-----------------+-----------------+



In [32]:
(
    weather.filter(F.year('date') == 2015)
    .filter(F.quarter('date') == 3)
    .select(
        F.when(
            F.col('weather') == 'rain', 1
            ).otherwise(0).alias('rain'))
    .agg(F.mean('rain'))
    .show()
)

+--------------------+
|           avg(rain)|
+--------------------+
|0.021739130434782608|
+--------------------+



In [33]:
(
    weather.withColumn(
        'year', F.year('date')
        )
    .select(
        F.when(
            F.col('precipitation') > 0, 1
            ).otherwise(0).alias('did_rain'), 'year'
            )
    .groupby('year')
    .agg(F.mean('did_rain'))
    .show()
)


+----+-------------------+
|year|      avg(did_rain)|
+----+-------------------+
|2012|0.48360655737704916|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2015|0.39452054794520547|
+----+-------------------+

